In [0]:
#from torchvision.references.detection import engine

In [1]:
!ls

sample_data


In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('drive/My Drive/object')

In [4]:
!ls datasets/CARPK_devkit/data/Images -l | wc -l

1449


In [0]:
!unzip -P hsieh_iccv17 datasets.zip

In [0]:
import torch
import torchvision
from carpk import Carpk
from torch.utils.data import DataLoader
from torchvision import transforms
import time
from engine import train_one_epoch,evaluate
import utils

In [0]:
carpklotdataset = Carpk('/content/drive/My Drive/object/datasets/CARPK_devkit', 'train', transform=transforms.Compose(
    [transforms.ToTensor()]))
data_loader=DataLoader(carpklotdataset,batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)
carpklotdataset_test = Carpk('/content/drive/My Drive/object/datasets/CARPK_devkit', 'test', transform=transforms.Compose(
    [transforms.ToTensor()]))
data_loader_test=DataLoader(carpklotdataset_test,batch_size=1,shuffle=False,collate_fn=utils.collate_fn)

In [8]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(num_classes=2,pretrained=False)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:07<00:00, 13.5MB/s]


In [9]:
torch.cuda.is_available()

True

In [10]:
!nvidia-smi

Sat Oct 26 15:58:30 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    29W / 149W |     11MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
device = torch.device("cuda")

In [12]:
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(original_name=FrozenBatchNorm2d)
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
       

In [0]:
params = [p for p in model.parameters() if p.requires_grad]

In [0]:
 optimizer = torch.optim.SGD(params, lr=0.02, momentum=0.9, weight_decay=1e-4)

In [0]:
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[8,11], gamma=0.1)


In [16]:
    # Training
    model_without_ddp = model
    print('Start training')
    start_time = time.time()
    for epoch in range(2):
        train_one_epoch(model, optimizer, data_loader, device, epoch, 1)
        lr_scheduler.step()
        if True:
            utils.save_on_master({
                'model': model_without_ddp.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                },
                os.path.join("./", 'model_{}.pth'.format(epoch)))

        # evaluate after every epoch
        evaluate(model, data_loader_test, device=device)

    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))

Start training
Epoch: [0]  [  0/495]  eta: 0:36:30  lr: 0.000060  loss: 1.9335 (1.9335)  loss_classifier: 0.7023 (0.7023)  loss_box_reg: 0.1615 (0.1615)  loss_objectness: 0.6939 (0.6939)  loss_rpn_box_reg: 0.3759 (0.3759)  time: 4.4251  data: 1.7017  max mem: 2486
Epoch: [0]  [  1/495]  eta: 0:24:43  lr: 0.000101  loss: 1.8736 (1.9036)  loss_classifier: 0.7015 (0.7019)  loss_box_reg: 0.1615 (0.2145)  loss_objectness: 0.6932 (0.6935)  loss_rpn_box_reg: 0.2115 (0.2937)  time: 3.0037  data: 0.8593  max mem: 2762
Epoch: [0]  [  2/495]  eta: 0:20:43  lr: 0.000141  loss: 1.8736 (1.8773)  loss_classifier: 0.7015 (0.7016)  loss_box_reg: 0.2309 (0.2200)  loss_objectness: 0.6939 (0.6937)  loss_rpn_box_reg: 0.2115 (0.2621)  time: 2.5227  data: 0.5737  max mem: 2762
Epoch: [0]  [  3/495]  eta: 0:47:59  lr: 0.000182  loss: 1.8248 (1.8523)  loss_classifier: 0.7011 (0.7012)  loss_box_reg: 0.1787 (0.2097)  loss_objectness: 0.6939 (0.6942)  loss_rpn_box_reg: 0.2025 (0.2472)  time: 5.8531  data: 3.9652 

NameError: ignored